In [363]:
netlist = """
***************************************
* auCdl Netlist
*
* Library Name: ML_DATA_LIB
* View Name : asdsa
* Top Cewll Name : sadasdsdf
* Netlisted On: FGerv, asdfasfd, 2 10:35:01 2022
***************************************
.INCLUDE $calibre_source_added_place
*.BIPOLAR
*.REST = 2000
*.MEGA
.PARAM

*.EQUIV xcvr_avdd_clk = n12
*+    loaddely = n13
*+    rst_n = n14
.SUBCKT BUFFER A Z gnd gnds vdd vdds
+ n1 n2 n3
+ n4 n5
* net net0 = nnnn
* net Z = /zzzz/asdasd00
M1 Z net0 vdd vdds mypmos w=5 l=1
* net A = /gggg/asdsa
M2 net0 A vdd vdds mypmos w=5 l=1
+ area=122 fin=34
+ sde=44
M3 Z net0 gnd gnds mynmos w=2 l=1
M4 net0 A gnd gnds mynmos w=2 l=1
.ENDS
"""

In [364]:
import pyparsing as _p

In [365]:
def parse_spectre(netlist_string):
    # newlines are part of the grammar, thus redifine the whitespaces without it
    ws = ' \t'
    _p.ParserElement.setDefaultWhitespaceChars(ws)

    # spectre netlist grammar definition
    EOL = _p.LineEnd().suppress() # end of line
    linebreak = _p.Suppress(_p.LineEnd() + '+') # breaking a line with backslash newline
    identifier=_p.Word(_p.alphanums+'_!<>/') # a name for...
    number=_p.Word(_p.nums + ".") # a number
    expression = _p.Word(_p.alphanums+'._*+-/()=!<>')
    net = expression # a net
    nets = _p.Group(_p.OneOrMore(net | linebreak)) # many nets
    cktname = identifier # name of a subcircuit
    cktname_end = _p.Keyword(".ENDS").suppress()
    instname = identifier
    instance = _p.Group(instname + nets + EOL).setResultsName('instance')
    int_net = _p.Suppress("* net") + identifier('key') + _p.Suppress("=") + identifier('value') + EOL
    int_nets = _p.Group(int_net).setResultsName('int_nets')
    subcircuit_content = _p.Group(_p.ZeroOrMore(instance | int_nets | EOL)).setResultsName('subnetlist')
    cir_net = identifier
    cir_nets = _p.Group(_p.OneOrMore(cir_net('net') | linebreak))
    ext_net = _p.Suppress(_p.Literal("*.EQUIV") | _p.Literal("*+"))  + identifier('value') + _p.Suppress("=") + identifier('key') + EOL
    ext_nets = _p.Group(ext_net).setResultsName('ext_nets')
    subcircuit = _p.Group(
        # matches subckt <name> <nets> <newline>
        _p.Keyword(".SUBCKT").suppress() + cktname('name') + cir_nets('nets') + EOL  
        # matches the content of the subcircuit
        + subcircuit_content
        # matches ends <name> <newline>
        + cktname_end + EOL).setResultsName('subcircuit')
    include = _p.Group(_p.Keyword(".INCLUDE") + ... + _p.Keyword(".PARAM"))
    star_line = _p.Group(_p.Combine(_p.Keyword("***") + ... + _p.Keyword("***")))
    netlist_element = subcircuit | _p.Suppress(include) | _p.Suppress(star_line) | ext_nets | EOL
    netlist = _p.ZeroOrMore(netlist_element) + _p.StringEnd()
    
    # parameters.setParseAction(handle_parameters)
    # instance.setParseAction(handle_instance)
    # subcircuit.setParseAction(handle_subcircuit)

    return netlist.parseString(netlist_string);

In [366]:
result1 = parse_spectre(netlist)

In [373]:
for ele in result1:
    print(ele)
    print('\n')

['xcvr_avdd_clk', 'n12']


['loaddely', 'n13']


['rst_n', 'n14']


['BUFFER', ['A', 'Z', 'gnd', 'gnds', 'vdd', 'vdds', 'n1', 'n2', 'n3', 'n4', 'n5'], [['net0', 'nnnn'], ['Z', '/zzzz/asdasd00'], ['M1', ['Z', 'net0', 'vdd', 'vdds', 'mypmos', 'w=5', 'l=1']], ['A', '/gggg/asdsa'], ['M2', ['net0', 'A', 'vdd', 'vdds', 'mypmos', 'w=5', 'l=1', 'area=122', 'fin=34', 'sde=44']], ['M3', ['Z', 'net0', 'gnd', 'gnds', 'mynmos', 'w=2', 'l=1']], ['M4', ['net0', 'A', 'gnd', 'gnds', 'mynmos', 'w=2', 'l=1']]]]




In [375]:
result1

ParseResults([ParseResults(['xcvr_avdd_clk', 'n12'], {'value': 'xcvr_avdd_clk', 'key': 'n12'}), ParseResults(['loaddely', 'n13'], {'value': 'loaddely', 'key': 'n13'}), ParseResults(['rst_n', 'n14'], {'value': 'rst_n', 'key': 'n14'}), ParseResults(['BUFFER', ParseResults(['A', 'Z', 'gnd', 'gnds', 'vdd', 'vdds', 'n1', 'n2', 'n3', 'n4', 'n5'], {'net': 'n5'}), ParseResults([ParseResults(['net0', 'nnnn'], {'key': 'net0', 'value': 'nnnn'}), ParseResults(['Z', '/zzzz/asdasd00'], {'key': 'Z', 'value': '/zzzz/asdasd00'}), ParseResults(['M1', ParseResults(['Z', 'net0', 'vdd', 'vdds', 'mypmos', 'w=5', 'l=1'], {})], {}), ParseResults(['A', '/gggg/asdsa'], {'key': 'A', 'value': '/gggg/asdsa'}), ParseResults(['M2', ParseResults(['net0', 'A', 'vdd', 'vdds', 'mypmos', 'w=5', 'l=1', 'area=122', 'fin=34', 'sde=44'], {})], {}), ParseResults(['M3', ParseResults(['Z', 'net0', 'gnd', 'gnds', 'mynmos', 'w=2', 'l=1'], {})], {}), ParseResults(['M4', ParseResults(['net0', 'A', 'gnd', 'gnds', 'mynmos', 'w=2', 'l